In [1]:
import tweepy
import pandas as pd
import numpy as np
import re
from googletrans import Translator
# from google_trans_new import google_translator  
import datetime
import time
# pd.options.display.max_columns = 999

In [124]:
# log = pd.read_csv("log_in.csv")
# consumer_key = log['key'][0]
# consumer_secret = log['key'][1]
# access_token = log['key'][2]
# access_token_secret = log['key'][3]
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth)
# keyword = 'ukraine'
# limit=5000
# # start_date = datetime.datetime(2022, 2, 24, 12, 00, 00)
# end_date = datetime.datetime(2023, 1, 23, 12, 00, 00)
# end_date = end_date.strftime("%Y-%m-%d")

# tweets = tweepy.Cursor(api.search_tweets, q=keyword, count=100, tweet_mode='extended', lang = 'ru', 
#                        result_type = 'mixed', until = end_date).items(limit)
# # tweets = api.search_tweets("ukraine", tweet_mode="extended", lang='ru')

# columns = ['User', 'Original Tweet']
# data = []

# #get tweet Id, location(city name, lang, long)
# for tweet in tweets:
#     data.append([tweet.user.screen_name, tweet.full_text])

# df = pd.DataFrame(data, columns=columns)

# print(df)


In [125]:
# count = 0

# keywords = ["ukraine", "war",'Украина','война','voyna']
# for response in tweepy.Paginator(client.search_all_tweets, 
#                                  query = 'ukraine -is:retweet lang:ru',
#                                  user_fields = ['username', 'public_metrics', 'description', 'location'],
#                                  tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
#                                  expansions = 'author_id',
#                                  start_time = '2022-02-18T00:00:00Z',
#                                  end_time = '2022-03-18T00:00:00Z',
#                                  max_results=500):
#     time.sleep(1)
#     if type(response) !=
#     if len(response) > 0:
#         for tweet in response:
#             if tweet not in selected_tweets:
#                 count += 1
#                 selected_tweets.append(tweet)
#             if count % 10 == 0:
#                 print(count)
#     if count >= 30:
#         break



# count = 0
# keywords = ["ukraine", "war", 'Украина', 'война', 'voyna']

# for page in tweepy.Paginator(client.search_all_tweets,
#                              query = 'ukraine -is:retweet lang:ru',
#                              user_fields=['username', 'public_metrics', 'description', 'location'],
#                              tweet_fields=['created_at', 'geo', 'public_metrics', 'text'],
#                              expansions='author_id',
#                              start_time='2022-02-18T00:00:00Z',
#                              end_time='2022-03-18T00:00:00Z',
#                              max_results=500):
#     time.sleep(1)
#     if len(page) > 0:
#         for item in page:  # Iterate through the items in the current page
#             if isinstance(item, dict):  # Check if the item is a dictionary
#                 tweet = item
#                 if tweet not in selected_tweets:
#                     count += 1
#                     selected_tweets.append(tweet)
#                     if count % 10 == 0:
#                         print(count)

#     if count >= 10:
#         break

In [126]:
# import tweepy
# import time

# count = 0
# selected_tweets_data = []
# selected_tweets_includes = []
# keywords = ["ukraine", "war", 'Украина', 'война', 'voyna']

# for page in tweepy.Paginator(client.search_all_tweets,
#                              query = 'ukraine -is:retweet lang:ru',
#                              user_fields=['username', 'public_metrics', 'description', 'location'],
#                              tweet_fields=['created_at', 'geo', 'public_metrics', 'text'],
#                              expansions='author_id',
#                              start_time='2022-02-18T00:00:00Z',
#                              end_time='2022-03-18T00:00:00Z',
#                              max_results=500):
#     time.sleep(1)

#     if len(page) > 0:
#         data = page.data
#         includes = page.includes
#         selected_tweets_data.append(data)
#         selected_tweets_includes.append(includes)
# #             tweet_includes.extend(includes.get('users', []))
#     count += 1
#     if count % 10 == 0:
#         print(count)

#     if count >= 10:
#         break


In [127]:
# result = []
# user_dict = {}
# # Loop through each response object
#     # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    
# for user in selected_tweets_includes:
#     user_dict[user.id] = {'username': user.username, 
#                           'followers': user.public_metrics['followers_count'],
#                           'tweets': user.public_metrics['tweet_count'],
#                           'description': user.description,
#                           'location': user.location
#                              }

# for tweet in selected_tweets_data:

#     # For each tweet, find the author's information

#     author_info = user_dict[tweet.author_id]

#     # Put all of the information we want to keep in a single dictionary for each tweet

#     result.append({'author_id': tweet.author_id, 
#                    'username': author_info['username'],
#                    'author_followers': author_info['followers'],
#                    'author_tweets': author_info['tweets'],
#                    'author_description': author_info['description'],                   
#                    'author_location': author_info['location'],                   
#                    'text': tweet.text,                   
#                    'created_at': tweet.created_at,                   
#                    'retweets': tweet.public_metrics['retweet_count'],                   
#                    'replies': tweet.public_metrics['reply_count'],                   
#                    'likes': tweet.public_metrics['like_count'],                   
#                    'quote_count': tweet.public_metrics['quote_count']
#                       })

# # Change this list of dictionaries into a dataframe
# df = pd.DataFrame(result)

In [102]:
# count = 0
# for response in selected_tweets:
#     count+=1
#     if isinstance(response, dict):
#         print("The object is a dictionary.")
#         print(count)
# #             print(response.keys())
# #             print('\n')

In [128]:
log = pd.read_csv("log_in.csv")
bearer_token = log['key'][4]


In [135]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [136]:
selected_tweets = []

In [ ]:
keywords = ["ukraine", "war",'Украина','война','voyna']
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = 'война -is:retweet lang:ru',
#                                  query=' OR '.join(keywords) + ' -is:retweet lang:ru',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2022-02-18T00:00:00Z',
                                 end_time = '2022-03-18T00:00:00Z',
                                 max_results=500):
    time.sleep(1)
    selected_tweets.append(response)
    if len(selected_tweets)%20==0:
        print(len(selected_tweets))
    if len(selected_tweets) > 5000:
        break



In [ ]:
result = []
user_dict = {}
# Loop through each response object
for response in selected_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id, 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [146]:
df.head()

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,951596246577418242,Fends86332387,582,23095,"Люблю «дедушку Ленина» (нежно), примерно как и...","USA, NE",@VictorKvert2008 К какой войне с Европой? \nУк...,2022-03-17 23:58:37+00:00,0,1,1,0
1,32421337,Maksim85,773,10422,"Everyone sees what you appear to be, few exper...",Chief Business Officer,#ukraine\n\n@IAPonomarenko это русская мать ищ...,2022-03-17 23:53:17+00:00,0,0,0,0
2,2830576792,vatnikliberal,497,912,American with a global perspective.,"New York, NY",@hrstnka @MFA_Ukraine Нет это предсмертные суд...,2022-03-17 23:49:42+00:00,0,1,10,0
3,1385590958,NuraddinAbbasov,1461,17611,Baku State University 🇦🇿🌈🇹🇷,None,🇺🇦После обстрела оккупантов горит крупнейший в...,2022-03-17 23:45:48+00:00,0,0,0,0
4,1148664081425018880,kain1952,73,42639,#RussiaIsATerroristState #armukrainenow #Мальц...,концлагерь зона колония,Украина германию франции\n#nowar #StopPutinNOW...,2022-03-17 23:43:58+00:00,0,0,0,0


In [147]:
df.shape

(104048, 12)

In [73]:
# Code to clean the dataset.

def clean_punc(line):
    temp = re.sub(r'[?|!|\'|"|#]',r'',line)
    temp = re.sub(r'[.|,|"|")|(|\|/]',r' ',temp)
    return  temp

def clean_html(line):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', line)
    return cleantext

def clean_space(line):
    return " ".join(line.split())


def clean_char(line):
    clean_ch = re.compile('[!@#$%^&*()_+-={};:\'\"\|<>,.///?`~।]', flags=re.I)
    return clean_ch.sub(r'',line)


def clean_non_bn_words(line):
    clean_en = re.compile('[a-z]', flags=re.I)
    clean_en_up = re.compile('[A-Z]', flags=re.I)
    clean_digit = re.compile('[0-9]', flags=re.I)
    line = clean_en.sub(r'',line)
    line = clean_en_up.sub(r'',line)
    return clean_digit.sub(r'',line)

def clean_emo(line):
    result = re.sub('[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)]+','',line)
    return result


def clean_all(line):
    line = clean_link(line)
    line = clean_html(line)
    line = clean_mention(line)
    line = clean_punc(line)
    line = clean_char(line)
    line = clean_space(line)
    return clean_emo(line)


# def float_to_int(val):
#     return int(val)


def clean_link(line):
    line = re.sub(r'https://\s?t\.co/\w+', '', line)
    return re.sub(r"(http\S+|t\.me\S+)", "", line)
    
    
def clean_mention(line):
    line = re.sub(r'#\w+', '', line)
    return re.sub(r'@\w+', '', line)    

In [74]:
# !pip install googletrans==4.0.0rc1

In [75]:
df_main = pd.read_csv('scrapped_tweets_ru.csv')

df_main.head()

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,0,951596246577418242,Fends86332387,582,23095,"Люблю «дедушку Ленина» (нежно), примерно как и...","USA, NE",@VictorKvert2008 К какой войне с Европой? \nУк...,2022-03-17 23:58:37+00:00,0,1,1,0
1,1,32421337,Maksim85,773,10422,"Everyone sees what you appear to be, few exper...",Chief Business Officer,#ukraine\n\n@IAPonomarenko это русская мать ищ...,2022-03-17 23:53:17+00:00,0,0,0,0
2,2,2830576792,vatnikliberal,497,912,American with a global perspective.,"New York, NY",@hrstnka @MFA_Ukraine Нет это предсмертные суд...,2022-03-17 23:49:42+00:00,0,1,10,0
3,3,1385590958,NuraddinAbbasov,1461,17611,Baku State University 🇦🇿🌈🇹🇷,NaN,🇺🇦После обстрела оккупантов горит крупнейший в...,2022-03-17 23:45:48+00:00,0,0,0,0
4,4,1148664081425018880,kain1952,73,42639,#RussiaIsATerroristState #armukrainenow #Мальц...,концлагерь зона колония,Украина германию франции\n#nowar #StopPutinNOW...,2022-03-17 23:43:58+00:00,0,0,0,0


In [76]:
df_main.shape

(86626, 13)

In [77]:
cleaned_text = []
for line in df_main['text']:
    temp = clean_all(line)
    cleaned_text.append(temp)

In [78]:
df_main['Cleaned Text'] = cleaned_text

In [79]:
check_line = 4
print(df_main['text'][check_line])
print('\n')
print(cleaned_text[check_line]) 


Украина германию франции
#nowar #StopPutinNOW #StopRussia #NOflyZoneUA  
#UkraineWar #UkraineRussianWar #Ukraine
russian #putin в россии рогозин кремля лукошенко белоруссии кадырова шойгу ссср путина- ФАШИСТЫ ПРОКЛЯТЫЕ ТЕРРОРИСТЫ МАРОДЁРЫ ИДИТЕ НА ХУЙ https://t.co/XCw8ROpDzd


Украина германию франции russian в россии рогозин кремля лукошенко белоруссии кадырова шойгу ссср путина ФАШИСТЫ ПРОКЛЯТЫЕ ТЕРРОРИСТЫ МАРОДЁРЫ ИДИТЕ НА ХУЙ


In [81]:
from googletrans import Translator
translator = Translator()
translated = translator.translate(text=df_main['Cleaned Text'][4], dest='en')
print(translated.text)

During the Putin war in Ukraine, civilian and residential buildings are bombing and also abduct civilians such as the mayors of the someone very frankly distorted perception when it comes to peacekeeping missions


In [82]:
translator = Translator()

translated_text = []

for text in df_main['Cleaned Text']:
    
    temp_text = translator.translate(text=text, dest='en')
    translated_text.append(temp_text.text)

ReadTimeout: The read operation timed out

In [ ]:
len(translated_text)

In [93]:
check_line = 9
print(df_main['text'][check_line])
print("\n")
print(cleaned_text[check_line]) 
print("\n")
print(translator.translate(text=df_main['Cleaned Text'][check_line], dest='en').text)
print("\n")
print(translator.translate(text=df_main['text'][check_line], dest='en').text)


#Ukraine　#Вторжение в Украину
Путин - массовый убийца.  Нет другого выбора, кроме как стереть Путина.  Мировой террористический вдохновитель.


в Украину Путин массовый убийца Нет другого выбора кроме как стереть Путина Мировой террористический вдохновитель


In Ukraine, Putin is a mass killer, there is no other choice but to erase Putin the world terrorist inspirer


#UKRAINE#Invasion in Ukraine
Putin is a mass killer.There is no other choice but to erase Putin.World terrorist inspirer.


In [95]:
df_main.shape

(86626, 14)

In [98]:
len(df_main['text']), len(df_main['Cleaned Text'])

(86626, 86626)

In [99]:
df_main.to_csv('cleaned_scrapped_tweets_ru.csv')